In [ ]:
#/content/drive/MyDrive/Data

**Import drive in order to load datasets**

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Import necessary libraries part 1 ##

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [3]:
# Import necessary torch libraries for main processes

In [13]:
import torch #for using tensors
import torch.nn as nn # creating neural network
import torch.nn.functional as F # recallable functions like relu,sigmoid and etc.
import torchvision # subpackage for vision models
import torchvision.transforms as transforms # image augmentation
import torch.optim as optim # optimisation functions like sgd, adam
from torch.utils.data import DataLoader # creating a data loader
from torchvision.transforms import ToTensor # converting image to tensor
import matplotlib.pyplot as plt 
from torchvision import datasets

In [14]:
torch.cuda.is_available()

False

In [16]:
batch_size=64

In [17]:
data_dir = '/content/drive/MyDrive/Data'
    
transform = transforms.Compose([transforms.RandomResizedCrop(150),transforms.ToTensor()])
    
train_set = datasets.ImageFolder(data_dir + '/seg_train',transform=transform)
test_set = datasets.ImageFolder(data_dir + '/seg_test',transform=transform)

    
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [18]:
len(train_loader)

220

In [10]:
data_dir

'/content/drive/MyDrive/Data'

In [19]:
for X,y in train_loader:
    print(X.shape)
    print(y)
    break

torch.Size([64, 3, 150, 150])
tensor([1, 4, 4, 5, 1, 1, 3, 0, 5, 2, 4, 1, 5, 3, 4, 1, 0, 2, 4, 3, 4, 1, 4, 5,
        4, 1, 5, 0, 3, 1, 5, 1, 5, 3, 4, 1, 0, 4, 2, 5, 1, 0, 0, 1, 3, 1, 3, 2,
        0, 5, 4, 2, 2, 2, 4, 3, 0, 1, 2, 1, 0, 4, 4, 0])


**Modelling**

In [ ]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=5,padding=2)
#         self.conv2 = nn.Conv2d(64,128,kernel_size=5,stride=2,padding=2)
#         self.pool = nn.MaxPool2d(2,2)
#         self.fc1 = nn.Linear(128*7*7,512)
#         self.fc2 = nn.Linear(512,512)
#         self.out = nn.Linear(512,10)
#         self.drop_out = nn.Dropout()
#     def forward(self, X):
#         X = self.conv1(X)
#         X = self.conv2(X)
#         X = self.pool(X)
#         X = X.reshape(X.size(0), -1)
#         X = self.drop_out(X)
#         X = F.relu(self.fc1(X))
#         X = F.relu(self.fc2(X))
#         out = self.out(X)
#         return X

In [30]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(64,128,kernel_size=5,stride=2,padding=2)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(128*37*37,512)
        self.fc2 = nn.Linear(512,512)
        self.out = nn.Linear(512,10)
        self.drop_out = nn.Dropout()
    def forward(self, X):
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.pool(X)
        X = X.reshape(X.size(0), -1)
        # X = self.drop_out(X)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        out = self.out(X)
        return X

In [31]:
(28-5+2*2)/2+1

14.5

In [32]:
model = NeuralNetwork()
# defining model

In [33]:
num_epochs = 6
num_classes = 10
learning_rate = 0.001

In [34]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [35]:
# Train the model
model.train()
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/6], Step [100/220], Loss: 4.8636, Accuracy: 28.12%
Epoch [1/6], Step [200/220], Loss: 4.9983, Accuracy: 25.00%
Epoch [2/6], Step [100/220], Loss: 1.9706, Accuracy: 46.88%
Epoch [2/6], Step [200/220], Loss: 1.7621, Accuracy: 59.38%
Epoch [3/6], Step [100/220], Loss: 1.9933, Accuracy: 53.12%
Epoch [3/6], Step [200/220], Loss: 1.8028, Accuracy: 54.69%
Epoch [4/6], Step [100/220], Loss: 2.1621, Accuracy: 56.25%
Epoch [4/6], Step [200/220], Loss: 2.2402, Accuracy: 54.69%
Epoch [5/6], Step [100/220], Loss: 2.2747, Accuracy: 51.56%
Epoch [5/6], Step [200/220], Loss: 1.4423, Accuracy: 67.19%
Epoch [6/6], Step [100/220], Loss: 2.0428, Accuracy: 59.38%
Epoch [6/6], Step [200/220], Loss: 1.7008, Accuracy: 62.50%


In [36]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 59.827070169604255 %


In [40]:
from bokeh.plotting import figure
from bokeh.io import show

In [43]:
from bokeh.models import LinearAxis, Range1d